# Per-Class Threshold Tuning for News Topic Classification

Optimizes per-label decision thresholds for `ContextNews/news-classifier` using the validation split of `ContextNews/labelled_articles`.

**Make sure to set Runtime > Change runtime type > T4 GPU**

In [ ]:
!pip install -q datasets transformers accelerate scikit-learn torch matplotlib

In [ ]:
from huggingface_hub import login
login()

## 1. Setup and Load Model + Dataset

In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np
import torch
from datasets import load_dataset
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer

MODEL_ID = "ContextNews/news-classifier"
DATASET_ID = "ContextNews/labelled_articles"

TOPICS = [
    "politics", "geopolitics", "conflict", "crime", "law", "business",
    "economy", "markets", "technology", "science", "health", "environment",
    "society", "education", "sports", "entertainment",
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID).to(device)
model.eval()

print(f"Model labels: {model.config.id2label}")

In [ ]:
val_ds = load_dataset(DATASET_ID, split="validation")
print(f"Validation set: {len(val_ds)} rows")

## 2. Run Inference on Validation Set

In [ ]:
def build_input_text(row):
    title = row.get("title") or ""
    summary = row.get("summary") or ""
    text = row.get("text") or ""
    text_excerpt = " ".join(text.split()[:300])
    return " ".join(p for p in [title, summary, text_excerpt] if p)


def preprocess(row):
    row["input_text"] = build_input_text(row)
    row["labels"] = [float(row[t] or 0) for t in TOPICS]
    return row


val_ds = val_ds.map(preprocess)

In [ ]:
BATCH_SIZE = 32

all_probs = []
all_labels = []

for i in range(0, len(val_ds), BATCH_SIZE):
    batch = val_ds[i : i + BATCH_SIZE]

    encodings = tokenizer(
        batch["input_text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        logits = model(**encodings).logits
        probs = torch.sigmoid(logits).cpu().numpy()

    all_probs.append(probs)
    all_labels.append(np.array(batch["labels"]))

    if (i // BATCH_SIZE) % 10 == 0:
        print(f"\r  Processed {min(i + BATCH_SIZE, len(val_ds))}/{len(val_ds)}", end="", flush=True)

print()

y_probs = np.concatenate(all_probs, axis=0)
y_true = np.concatenate(all_labels, axis=0)

print(f"y_true shape:  {y_true.shape}")
print(f"y_probs shape: {y_probs.shape}")

## 3. Baseline Metrics (threshold = 0.5)

In [ ]:
y_pred_baseline = (y_probs >= 0.5).astype(int)

baseline_metrics = {
    "f1_micro": f1_score(y_true, y_pred_baseline, average="micro", zero_division=0),
    "f1_macro": f1_score(y_true, y_pred_baseline, average="macro", zero_division=0),
    "precision": precision_score(y_true, y_pred_baseline, average="micro", zero_division=0),
    "recall": recall_score(y_true, y_pred_baseline, average="micro", zero_division=0),
}

print("Baseline metrics (threshold = 0.5):")
for k, v in baseline_metrics.items():
    print(f"  {k}: {v:.4f}")

print("\nPer-class F1 (baseline):")
baseline_per_class = {}
for i, topic in enumerate(TOPICS):
    f1 = f1_score(y_true[:, i], y_pred_baseline[:, i], zero_division=0)
    baseline_per_class[topic] = f1
    print(f"  {topic:15s} {f1:.4f}")

## 4. Per-Class Threshold Sweep

In [ ]:
thresholds_range = np.arange(0.05, 0.96, 0.01)

optimal_thresholds = {}
f1_curves = {}  # topic -> list of (threshold, f1)

for i, topic in enumerate(TOPICS):
    best_t = 0.5
    best_f1 = 0.0
    curve = []

    for t in thresholds_range:
        preds = (y_probs[:, i] >= t).astype(int)
        f1 = f1_score(y_true[:, i], preds, zero_division=0)
        curve.append((t, f1))
        if f1 > best_f1:
            best_f1 = f1
            best_t = float(round(t, 2))

    optimal_thresholds[topic] = best_t
    f1_curves[topic] = curve
    print(f"  {topic:15s}  threshold={best_t:.2f}  F1={best_f1:.4f}  (baseline F1={baseline_per_class[topic]:.4f})")

print("\nOptimal thresholds:")
print(json.dumps(optimal_thresholds, indent=2))

## 5. Evaluate with Tuned Thresholds

In [ ]:
y_pred_tuned = np.zeros_like(y_probs, dtype=int)
for i, topic in enumerate(TOPICS):
    y_pred_tuned[:, i] = (y_probs[:, i] >= optimal_thresholds[topic]).astype(int)

tuned_metrics = {
    "f1_micro": f1_score(y_true, y_pred_tuned, average="micro", zero_division=0),
    "f1_macro": f1_score(y_true, y_pred_tuned, average="macro", zero_division=0),
    "precision": precision_score(y_true, y_pred_tuned, average="micro", zero_division=0),
    "recall": recall_score(y_true, y_pred_tuned, average="micro", zero_division=0),
}

print("Tuned metrics:")
for k, v in tuned_metrics.items():
    print(f"  {k}: {v:.4f}")

print("\n" + "=" * 45)
print(f"  Baseline macro F1: {baseline_metrics['f1_macro']:.4f}")
print(f"  Tuned macro F1:    {tuned_metrics['f1_macro']:.4f}")
improvement = tuned_metrics["f1_macro"] - baseline_metrics["f1_macro"]
print(f"  Improvement:       {improvement:+.4f}")
print("=" * 45)

## 6. Save Thresholds to Disk

In [ ]:
OUTPUT_PATH = "news_classifier_thresholds.json"

with open(OUTPUT_PATH, "w") as f:
    json.dump(optimal_thresholds, f, indent=2)

print(f"Thresholds saved to {OUTPUT_PATH}")

## 7. F1 vs Threshold Curves (3 Worst Performing Labels)

In [ ]:
sorted_by_f1 = sorted(baseline_per_class.items(), key=lambda x: x[1])
worst_3 = [topic for topic, _ in sorted_by_f1[:3]]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, topic in zip(axes, worst_3):
    curve = f1_curves[topic]
    ts = [t for t, _ in curve]
    f1s = [f for _, f in curve]
    best_t = optimal_thresholds[topic]
    best_f1 = max(f1s)

    ax.plot(ts, f1s, linewidth=1.5)
    ax.axvline(x=0.5, color="gray", linestyle="--", alpha=0.5, label="default (0.5)")
    ax.axvline(x=best_t, color="red", linestyle="--", alpha=0.7, label=f"optimal ({best_t:.2f})")
    ax.scatter([best_t], [best_f1], color="red", zorder=5)
    ax.set_title(topic)
    ax.set_xlabel("Threshold")
    ax.set_ylabel("F1")
    ax.set_ylim(0, 1)
    ax.legend(fontsize=8)

plt.suptitle("F1 vs Threshold - 3 Worst Performing Labels", y=1.02)
plt.tight_layout()
plt.show()